# Urychlení deduplikace pomocí vektorizovaných operací místo for-cyklů

# Příprava

Začátek je v kondenzované podobě stejný jako u `2-99-final.ipynb`, komentáře případně viz tam.

In [1]:
def flatten(lst, accumulator=None):
    if accumulator is None:
        accumulator = []
    for item in lst:
        if isinstance(item, list):
            flatten(item, accumulator)
        else:
            accumulator.append(item)
    return accumulator

In [2]:
from nltk.corpus import abc

docs = []
for cat in abc.fileids():
    for doc in abc.paras(fileids=cat):
        docs.append((cat[:-4], flatten(doc)))

In [3]:
import random

def make_dirty(docs):
    random.seed(100)
    dirty_docs = []
    for cat, doc in docs:
        if random.random() < .1:
            dirty_doc = []
            for token in doc:
                if random.random() > .05:
                    dirty_doc.append(token)
            dirty_docs.append((cat, dirty_doc))
        dirty_docs.append((cat, doc))
    random.shuffle(dirty_docs)
    return dirty_docs

In [4]:
dirty_docs = make_dirty(docs)
len(docs), len(dirty_docs)

(3189, 3492)

In [5]:
import gensim as gs

dirty_dictionary = gs.corpora.Dictionary(d for _, d in dirty_docs)
dirty_corpus = [dirty_dictionary.doc2bow(d) for _, d in dirty_docs]
tfidf_model = gs.models.TfidfModel(dirty_corpus)
dirty_corpus_tfidf = tfidf_model[dirty_corpus]
matrix_similarity = gs.similarities.MatrixSimilarity(dirty_corpus_tfidf)

# Kompletní pole / tabulka / matice podobností

A teď -- místo abychom podobnosti procházeli ve dvou vnořených for-cyklech, načteme je všechny do jedné dvourozměrné tabulky (přísně vzato *pole*, angl. *array*) pomocí knihovny `numpy` a třídy `ndarray`:

In [6]:
import numpy as np

similarity_table = np.array(matrix_similarity)
similarity_table

array([[1.00000012e+00, 2.35655364e-02, 3.09740119e-02, ...,
        2.02843230e-02, 2.59479042e-04, 6.02959772e-04],
       [2.35655364e-02, 1.00000000e+00, 1.13137057e-02, ...,
        3.09589151e-02, 1.60119273e-02, 1.49210449e-02],
       [3.09740119e-02, 1.13137057e-02, 1.00000000e+00, ...,
        2.61870120e-03, 1.29912505e-02, 1.95524725e-03],
       ...,
       [2.02843230e-02, 3.09589151e-02, 2.61870120e-03, ...,
        1.00000012e+00, 3.61138693e-04, 6.46825973e-03],
       [2.59479042e-04, 1.60119273e-02, 1.29912505e-02, ...,
        3.61138693e-04, 1.00000000e+00, 3.33650759e-03],
       [6.02959772e-04, 1.49210449e-02, 1.95524725e-03, ...,
        6.46825973e-03, 3.33650759e-03, 9.99999940e-01]], dtype=float32)

In [7]:
type(similarity_table)

numpy.ndarray

In [8]:
# objekty typu array můžou teoreticky mít libovolný
# počet rozměrů, tento náš je dvourozměrná tabulka
similarity_table.ndim

2

In [9]:
# počet řádků a sloupců
similarity_table.shape

(3492, 3492)

Pro trošku přehlednější a lépe čitelné zobrazení můžeme `ndarray` pomocí knihovny `pandas` převést na tzv. `DataFrame`:

In [10]:
import pandas as pd

df = pd.DataFrame(similarity_table)
df

,0,1,2,3,4,5,6,7,8,9,...,3482,3483,3484,3485,3486,3487,3488,3489,3490,3491
0,1.000000,0.023566,0.030974,0.006135,0.048488,0.014640,0.001964,0.004974,0.034417,0.007292,...,0.005511,0.004699,0.015097,0.029083,0.002398,0.014274,0.000493,0.020284,0.000259,0.000603
1,0.023566,1.000000,0.011314,0.017849,0.005569,0.007815,0.015914,0.007599,0.008650,0.016335,...,0.007180,0.012245,0.006294,0.029293,0.005592,0.007113,0.006032,0.030959,0.016012,0.014921
2,0.030974,0.011314,1.000000,0.002780,0.013671,0.006871,0.011316,0.005081,0.034592,0.008189,...,0.001082,0.001999,0.002476,0.028271,0.002103,0.028489,0.003087,0.002619,0.012991,0.001955
3,0.006135,0.017849,0.002780,1.000000,0.007520,0.013662,0.002640,0.006594,0.003020,0.008687,...,0.003500,0.005177,0.011035,0.009338,0.003348,0.007391,0.001788,0.003293,0.025259,0.015452
4,0.048488,0.005569,0.013671,0.007520,1.000000,0.027734,0.006407,0.010601,0.006067,0.008334,...,0.067801,0.011323,0.049884,0.006692,0.005013,0.020604,0.036756,0.016701,0.003415,0.008411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3487,0.014274,0.007113,0.028489,0.007391,0.020604,0.007797,0.029100,0.010674,0.036847,0.022709,...,0.009055,0.019783,0.056052,0.014436,0.018465,1.000000,0.191710,0.025678,0.022279,0.020921
3488,0.000493,0.006032,0.003087,0.001788,0.036756,0.004267,0.001284,0.009257,0.004288,0.024809,...,0.005657,0.007473,0.027228,0.018072,0.017672,0.191710,1.000000,0.018912,0.025898,0.005283
3489,0.020284,0.030959,0.002619,0.003293,0.016701,0.004636,0.004173,0.010352,0.010198,0.027463,...,0.026872,0.019128,0.009421,0.013752,0.059230,0.025678,0.018912,1.000000,0.000361,0.006468
3490,0.000259,0.016012,0.012991,0.025259,0.003415,0.000212,0.027288,0.010643,0.013706,0.009627,...,0.009945,0.000705,0.031996,0.002664,0.021818,0.022279,0.025898,0.000361,1.000000,0.003337


A případně si vyříznout jen část tabulky jako náhled (zde vzájemné podobnosti mezi prvními deseti dokumenty):

In [11]:
df.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.023566,0.030974,0.006135,0.048488,0.014640,0.001964,0.004974,0.034417,0.007292
1,0.023566,1.000000,0.011314,0.017849,0.005569,0.007815,0.015914,0.007599,0.008650,0.016335
2,0.030974,0.011314,1.000000,0.002780,0.013671,0.006871,0.011316,0.005081,0.034592,0.008189
3,0.006135,0.017849,0.002780,1.000000,0.007520,0.013662,0.002640,0.006594,0.003020,0.008687
4,0.048488,0.005569,0.013671,0.007520,1.000000,0.027734,0.006407,0.010601,0.006067,0.008334
5,0.014640,0.007815,0.006871,0.013662,0.027734,1.000000,0.008127,0.003067,0.001164,0.005920
6,0.001964,0.015914,0.011316,0.002640,0.006407,0.008127,1.000000,0.007783,0.026696,0.014562
7,0.004974,0.007599,0.005081,0.006594,0.010601,0.003067,0.007783,1.000000,0.005986,0.019910
8,0.034417,0.008650,0.034592,0.003020,0.006067,0.001164,0.026696,0.005986,1.000000,0.020656
9,0.007292,0.016335,0.008189,0.008687,0.008334,0.005920,0.014562,0.019910,0.020656,1.000000


# Nevýhody for-cyklů v Pythonu

Co tím získáme? For-cykly v Pythonu jsou relativně pomalé. Python je tzv. dynamický jazyk, jakákoli proměnná může obsahovat jakoukoli hodnotu -- číslo, řetězec, kolekci... Takže při jakýchkoli operacích musí Python nejprve zkontrolovat, o jaký typ hodnoty se jedná, jestli danou operaci podporuje atp., než se operaci vůbec pokusí vykonat.

Ve for-cyklu se zátěž těchhle kontrol nasčítá. Přitom v běžných případech obsahují proměnné v různých opakováních for-cyklu vždycky stejné typy hodnot (např. ve `for i in range(100): ...` bude `i` vždy číslo), takže ideální by bylo, kdyby si Python ve for-cyklu mohl tyhle kontroly ušetřit, respektive odbýt jen jednou hned napoprvé a při dalších opakováních už se spolehnout na to, že `i` bude vždy číslo a jde s ním tedy např. sčítat.

Jenže v praxi se na to spolehnout nelze, nic nám totiž nebrání napsat třeba takovýhle kód, kde se v jednom bodě typ hodnoty v proměnné `i` najednou změní:

In [12]:
for i in range(10):
    if i > 5:
        i = str(i)
    print(i**2)

0
1
4
9
16
25


TypeError: unsupported operand type(s) for ** or pow(): 'str' and 'int'

Takže Pythonu nezbývá, než to všechno hlídat pokaždé. Nemůže jen tak slepě doufat, že když v `i` bylo jednou číslo, bude v ní číslo pokaždé, a tudíž rovnou provést operaci `i**2`. Musí typ `i` pokaždé zkontrolovat, aby mohl v případě nesrovnalosti vyhodit příslušnou chybu (viz výše).

# Vektorizace: hromadné operace bez for-cyklů

Naštěstí existují způsoby, jak tento typ hromadného repetitivního zpracování dat v Pythonu urychlit. Např. právě pomocí knihovny `numpy` se můžeme for-cyklům úplně vyhnout: data ukládáme do homogenních polí, která (většinou) obsahují stejné typy hodnot, a aplikujeme na ně tzv. vektorizované operace:

In [13]:
pole = np.arange(10)
pole

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [14]:
# normální součet
0 + 10

10

In [15]:
# vektorizovaný součet
pole + 10

array([10, 11, 12, 13, 14, 15, 16, 17, 18, 19])

Vektorizované operace se aplikují na všechny prvky pole najednou, takže kontrolu kompatibility typů před provedením operace v tomto případě stačí skutečně provést jen jednou. Python si v příkladu výše ohlídá, že celé pole je číselné a přičítáme k němu číslo, a pak operaci provede ve zrychleném režimu, kdy nemusí kontrolu znovu opakovat pro každé individuální číslo v poli, jako by to musel dělat ve for-cyklu.

# Nevýhody vektorizace

Jaké má tento přístup naopak nevýhody? Např. tu, že je potřeba kompletní data načíst do paměti najednou, aby s nimi šlo manipulovat naráz. Vnořené for-cykly pracovaly s jednou hodnotou po druhé, což zabírá velmi málo paměti. Naopak s tímto přístupem se nám může snadno stát, že u velkého korpusu nebudeme schopni celou tabulku ani vytvořit. V takovém případě by pak mělo smysl zvolit kompromis a zpracovávat data v nějakých rozumně velkých várkách. Pak nám sice jeden vnější for-cyklus zbyde, ale každé jeho opakování zpracuje celou várku hodnot, čímž si pořád nějaký čas ušetříme.

Další nevýhodou pochopitelně je, že vektorizované programování vyžaduje cvik a kreativitu, není vždy snadné a přímočaré "normální" program přeložit do vektorizované podoby. A v jistém smyslu je výsledný kód hůře čitelný a srozumitelný než staré dobré for-cykly, byť jde samozřejmě do jisté míry o zvyk.

# Vektorizovaný algoritmus deduplikace

## Nástin

Když teď tedy máme kompletní tabulku podobností, jakým způsobem bychom ji mohli pomocí těchto globálních vektorizovaných operací přetavit v sérii identifikátorů dokumentů k ponechání či odstranění? Nabízím následující řešení (možná to jde i jednodušeji?) -- připomeňme si ještě jednou, jak tabulka vypadá:

In [16]:
df.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,1.000000,0.023566,0.030974,0.006135,0.048488,0.014640,0.001964,0.004974,0.034417,0.007292
1,0.023566,1.000000,0.011314,0.017849,0.005569,0.007815,0.015914,0.007599,0.008650,0.016335
2,0.030974,0.011314,1.000000,0.002780,0.013671,0.006871,0.011316,0.005081,0.034592,0.008189
3,0.006135,0.017849,0.002780,1.000000,0.007520,0.013662,0.002640,0.006594,0.003020,0.008687
4,0.048488,0.005569,0.013671,0.007520,1.000000,0.027734,0.006407,0.010601,0.006067,0.008334
5,0.014640,0.007815,0.006871,0.013662,0.027734,1.000000,0.008127,0.003067,0.001164,0.005920
6,0.001964,0.015914,0.011316,0.002640,0.006407,0.008127,1.000000,0.007783,0.026696,0.014562
7,0.004974,0.007599,0.005081,0.006594,0.010601,0.003067,0.007783,1.000000,0.005986,0.019910
8,0.034417,0.008650,0.034592,0.003020,0.006067,0.001164,0.026696,0.005986,1.000000,0.020656
9,0.007292,0.016335,0.008189,0.008687,0.008334,0.005920,0.014562,0.019910,0.020656,1.000000


Diagonála s hodnotami pro shodu dokumentu se sebou samým ($\approx 1$) představuje osu symetrie hodnot: podobnost dokumentu 5 s dokumentem 1 je stejná jako podobnost dokumentu 1 s dokumentem 5. První z nich je pod diagonálou, druhá nad diagonálou, a jestli je ta hodnota větší než 0.9, tak považujeme dokumenty 1 a 5 za duplikáty.

Jenže kdybychom odfiltrovali všechny řádky, které obsahují hodnoty $\gt 0.9$, tak bychom z výsledného korpusu vyhodili jak dokument 1, tak dokument 5, a z této duplicitní sady by tedy v datech nezbyla ani jedna položka, což nechceme. Podle zadání chceme v korpusu první ze sady zanechat (1) a další vyhodit (5). Jak toho docílit?

Můžeme využít té symetrie a trojúhelník nad diagonálou "zahodit". Tím nepřijdeme o žádné informace -- je v něm zrcadlově totéž co pod diagonálou -- ale tabulku převedeme do podoby, kde pro každý dokument budeme v jeho řádku mít hodnoty podobnosti jen pro dokumenty, které mu v korpusu předcházejí.

Např. v řádku pro dokument 1 bude jen podobnost s dokumentem 0 -- ostatní už jsou v tabulce nad diagonálou, takže budou vynulované. Tím pádem na podobnost s dokumentem 5 nenarazíme, a tudíž dokument 1 na jejím základě ani nevyřadíme. Naopak u dokumentu 5 bude součástí řádku i podobnost s dříve se vyskytujícím dokumentem 1, takže ten už při hodnotě $\gt 0.9$ vyřadíme.

## Vizualizovaný příklad

Ukažme si to vizuálně na uměle vytvořeném příkladu. V původní verzi tabulky podobností je v každém řádku minimálně jedna hodnota $\gt 0.9$, a to na diagonále, a každá duplicita je vyznačena (viz červeně zvýrazněné buňky níže). V této podobě tedy nelze jako duplikáty jednoduše označit všechny řádky, které obsahují alespoň jednu hodnotu $\gt 0.9$, protože to by platilo o všech.

In [17]:
ex1 = pd.DataFrame(
      [[1.        , 0.0237979 , 0.03096479, 0.00613627, 0.04849318, 0.0237979 , 0.01462847],
       [0.0237979 , 0.99999976, 0.01143176, 0.0179712 , 0.00563036, 0.94      , 0.00789504],
       [0.03096479, 0.01143176, 1.0000001 , 0.00278166, 0.01367155, 0.01143176, 0.00685627],
       [0.00613627, 0.0179712 , 0.00278166, 0.9999999 , 0.00752222, 0.0179712 , 0.01366599],
       [0.04849318, 0.00563036, 0.01367155, 0.00752222, 0.9999998 , 0.00563036, 0.02773733],
       [0.0237979 , 0.94      , 0.01143176, 0.0179712 , 0.00563036, 0.99999976, 0.00789504],
       [0.01462847, 0.00789504, 0.00685627, 0.01366599, 0.02773733, 0.00789504, 1.        ]]
)

def hi_lo(val):
    if val == 0:
        return "color: gray"
    elif val > 0.9:
        return "color: red"
    else:
        return ""

ex1.style.applymap(hi_lo)

,0,1,2,3,4,5,6
0,1.000000,0.023798,0.030965,0.006136,0.048493,0.023798,0.014628
1,0.023798,1.000000,0.011432,0.017971,0.005630,0.940000,0.007895
2,0.030965,0.011432,1.000000,0.002782,0.013672,0.011432,0.006856
3,0.006136,0.017971,0.002782,1.000000,0.007522,0.017971,0.013666
4,0.048493,0.005630,0.013672,0.007522,1.000000,0.005630,0.027737
5,0.023798,0.940000,0.011432,0.017971,0.005630,1.000000,0.007895
6,0.014628,0.007895,0.006856,0.013666,0.027737,0.007895,1.000000


V praxi ale nemůžeme a ani nechceme ten vrchní trojúhelník jen tak zahodit -- celý princip `numpy` a vektorizace je postavený na práci s poli, které mají pravidelné rozměry. Ale můžeme ho vynulovat, což je pro naše účely ekvivalentní.

Když se nám podaří vynulovat všechny hodnoty v tabulce od diagonály výš, tak nám v tabulce zbyde jediný řádek s hodnotou $\gt 0.9$, a to řádek 5, který skutečně chceme vyřadit, jak jsme rozebrali výše:

In [18]:
ex2 = pd.DataFrame(
      [[0.        , 0.        , 0.        , 0.        , 0.        , 0.        , 0.],
       [0.0237979 , 0.        , 0.        , 0.        , 0.        , 0.        , 0.],
       [0.03096479, 0.01143176, 0.        , 0.        , 0.        , 0.        , 0.],
       [0.00613627, 0.0179712 , 0.00278166, 0.        , 0.        , 0.        , 0.],
       [0.04849318, 0.00563036, 0.01367155, 0.00752222, 0.        , 0.        , 0.],
       [0.0237979 , 0.94      , 0.01143176, 0.0179712 , 0.00563036, 0.        , 0.],
       [0.01462847, 0.00789504, 0.00685627, 0.01366599, 0.02773733, 0.00789504, 0.]]
)
ex2.style.applymap(hi_lo)

,0,1,2,3,4,5,6
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.023798,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.030965,0.011432,0.000000,0.000000,0.000000,0.000000,0.000000
3,0.006136,0.017971,0.002782,0.000000,0.000000,0.000000,0.000000
4,0.048493,0.005630,0.013672,0.007522,0.000000,0.000000,0.000000
5,0.023798,0.940000,0.011432,0.017971,0.005630,0.000000,0.000000
6,0.014628,0.007895,0.006856,0.013666,0.027737,0.007895,0.000000


# Implementace

## Vynulování od diagonály dál

Jak analogicky vynulovat tu naši tabulku, aniž bychom to pracně museli dělat ručně, nebo pomocí for-cyklu, čímž bychom se připravili o hlavní výhodu vektorizace -- rychlost?

Stačí vytvořit pole o stejných rozměrech, které má pod diagonálou jedničky a všude jinde nuly, a tímto polem tu naši tabulku podobností vynásobit. Ve chvíli, kdy mají dvě pole stejné rozměry, zafunguje to jako "maska", tj. vynásobí se navzájem buňky na stejných pozicích.

In [19]:
# takhle můžeme vytvořit tzv. trojúhelníkovou matici
# o rozměru 5×5
np.tri(5, 5)

array([[1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0.],
       [1., 1., 1., 1., 1.]])

In [20]:
# v našem případě chceme nuly i na hlavní diagonále,
# takže hranici, kam až sahají jedničky, chceme posunout
# o jednu pozici dolů, čehož docílíme tím třetím
# argumentem -1
np.tri(5, 5, -1)

array([[0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [1., 1., 1., 0., 0.],
       [1., 1., 1., 1., 0.]])

In [21]:
# analogickou trojúhelníkovou masku vytvoříme v rozměrech
# odpovídajících tabulce podobností, a tabulku jí rovnou
# přenásobíme
similarity_table *= np.tri(*similarity_table.shape, -1)
# numpy se snaží šetřit paměť, takže nové hodnoty přeuloží
# na stejné místo v paměti, kde bylo původní pole; vzhledem
# k tomu, že náš DataFrame je vlastně jen vylepšeným pohledem
# na data v tomto původním poli, tak se změna rovnou promítne
# i do něj
df.iloc[:10, :10]

,0,1,2,3,4,5,6,7,8,9
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
1,0.023566,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
2,0.030974,0.011314,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
3,0.006135,0.017849,0.002780,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
4,0.048488,0.005569,0.013671,0.007520,0.000000,0.000000,0.000000,0.000000,0.000000,0.0
5,0.014640,0.007815,0.006871,0.013662,0.027734,0.000000,0.000000,0.000000,0.000000,0.0
6,0.001964,0.015914,0.011316,0.002640,0.006407,0.008127,0.000000,0.000000,0.000000,0.0
7,0.004974,0.007599,0.005081,0.006594,0.010601,0.003067,0.007783,0.000000,0.000000,0.0
8,0.034417,0.008650,0.034592,0.003020,0.006067,0.001164,0.026696,0.005986,0.000000,0.0
9,0.007292,0.016335,0.008189,0.008687,0.008334,0.005920,0.014562,0.019910,0.020656,0.0


## Identifikace řádků s podobnostmi $\gt 0.9$

Následně už si můžeme zjistit, kde jsou hodnoty podobnosti $\gt 0.9$, protože víme, že každý pár vysoce podobných dokumentů na nás vyskočí jen jednou (protože zrcadlené hodnoty nad diagonálou jsme vynulovali). Operace znovu proběhne vektorizovaně pro celou tabulku naráz:

In [22]:
similarity_table > .9

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

Nás ale konkrétně zajímá, které *řádky* obsahují *alespoň jednu* hodnotu $\gt 0.9$. To se můžeme dozvědět pomocí funkce `numpy.any()`, která dohledá, zda pole obsahuje nějakou hodnotu `True`:

In [23]:
np.any(similarity_table > .9)

True

Hmm... Teď jsme se dozvěděli, že v celé tabulce je minimálně jedna hodnota $\gt 0.9$. To je sice pěkné, ale pro náš aktuální úkol nepříliš užitečné. Jak vyjádřit, že nechceme jeden celkový výsledek pro celou tabulku, ale dílčí výsledek pro každý řádek?

## Osy vícerozměrných polí a argument `axis`

K tomu slouží argument `axis`, který specifikuje, po které ose se hodnoty procházejí. První osa (`axis=0`) odkazuje k řádkům, druhá (`axis=1`) ke sloupcům, což si lépe ověříme na matici, která není čtvercová:

In [24]:
arr = np.array([
    [.1, .1, .1],
    [.91, .1, .1]
])
arr

array([[0.1 , 0.1 , 0.1 ],
       [0.91, 0.1 , 0.1 ]])

In [25]:
# počet os/dimenzí ("number of dimensions")
arr.ndim

2

In [26]:
# délky obou os/dimenzí
arr.shape

(2, 3)

První osa jsou řádky, druhá sloupce, a skutečně máme v poli `arr` dva řádky a tři sloupce, takže to sedí.

Podobně když chceme z matice vytáhnout jednu konkrétní hodnotu, tak je potřeba zadat dva indexy, a první zvolí řádek a druhý sloupec:

In [27]:
arr[1, 0]

0.91

Tohle pořadí koneckonců naznačuje i reprezentace pole -- připomíná seznam seznamů, přičemž každý vnořený seznam je jeden řádek a obsahuje hodnoty pro jednotlivé sloupce.

In [28]:
lst = arr.tolist()
lst

[[0.1, 0.1, 0.1], [0.91, 0.1, 0.1]]

Abychom z takového seznamu seznamů vytáhli jednu konkrétní hodnotu, musíme provést dvě indexační operace, přičemž první specifikuje řádek a druhá pak v rámci zvoleného řádku sloupec:

In [29]:
lst[1]

[0.91, 0.1, 0.1]

In [30]:
lst[1][0]

0.91

A se stejným pořadím os počítají i argumenty `axis=...`. Vyskytují se u různých funkcí, kde se hodí mít možnost vybrat, ve kterém směru má vektorizace proběhnout.

## Zmatky okolo sémantiky argumentu `axis`

Jejich sémantika ale může být trochu matoucí. Mohlo by se zdát, že `axis=0` znamená, že se funkce aplikuje po řádcích a dostaneme tedy výsledek pro každý řádek, a `axis=1` zas vrátí výsledek pro každý sloupec. Jenže opak je pravdou:

In [31]:
# axis=0, tj. osa řádků, ale dostaneme tři výsledky,
# jeden pro každý sloupec
np.any(arr > .9, axis=0)

array([ True, False, False])

In [32]:
# axis=1, tj. osa sloupců, ale dostaneme dva výsledky,
# jeden pro každý řádek
np.any(arr > .9, axis=1)

array([False,  True])

Jinými slovy -- člověk by možná čekal, že počet výsledků při aplikaci funkce po dané ose bude odpovídat délce osy, ale není tomu tak:

In [33]:
for axis, axis_len in enumerate(arr.shape):
    result_len = len(np.any(arr > .9, axis=axis))
    print(f"{axis = }, {axis_len = }, {result_len = }")

axis = 0, axis_len = 2, result_len = 3
axis = 1, axis_len = 3, result_len = 2


## Intuice za tím, proč argument `axis` funguje, jak funguje

Jaká je tedy motivace za tím, že je to (z tohoto pohledu) naopak? Představme si to tak, že **řádky jsou sice horizontální, ale osa, která je popisuje, je na ně kolmá, tj. vertikální**, takže aplikovat funkci po směru této osy vlastně znamená aplikovat ji na sloupce.

Nebo jiná mnemotechnická pomůcka: **argument `axis` specifikuje, kterou osu při operaci "zploštit", odstranit**. Takže pokud chceme výsledky po řádcích, nezadáme do argumentu `axis` osu řádků, ale osu sloupců, kterou tímto odstraníme / zploštíme. Můžeme si to znázornit třeba takto:

In [34]:
shape = ["rows", "columns"]
print(f"osy před operací: {shape}")
axis = 1
print(f"argument {axis = } ukazuje na osu {shape[axis]!r}")
print("  -> tato osa bude zploštěna / odstraněna")
shape.pop(axis)
print(f"osy po operaci: {shape}")

osy před operací: ['rows', 'columns']
argument axis = 1 ukazuje na osu 'columns'
  -> tato osa bude zploštěna / odstraněna
osy po operaci: ['rows']


Když tuto úvahu generalizujeme i na více než dvourozměrná pole, tak je potřeba do argumentu `axis` zadat n-tici všch os, které nechceme ve výsledku zachovat, což bychom mohli formulovat např. takhle:

In [35]:
for axis, axis_len in enumerate(arr.shape):
    # n-tice všech os kromě té, kterou chceme ve výstupu zachovat
    collapsed_axes = tuple(x for x in range(arr.ndim) if x != axis)
    result_len = len(np.any(arr > .9, axis=collapsed_axes))
    print(f"{axis = }, {collapsed_axes = } {axis_len = }, {result_len = }")

axis = 0, collapsed_axes = (1,) axis_len = 2, result_len = 2
axis = 1, collapsed_axes = (0,) axis_len = 3, result_len = 3


Teď už hodnoty `axis_len` a `result_len` v rámci jednoho řádku sedí.

## Pozor na záměnu os u čtvercových matic!

Nejzáludnější je to u čtvercových matic jako je naše `similarity_table`, tam bude totiž počet výsledků stejný v obou směrech, ale v jednom z nich to budou úplně jiné výsledky, než jsme zamýšleli. Tak, jak jsme si tabulku připravili, nám výsledky po řádcích označují dokumenty k odstranění...

In [36]:
np.any(similarity_table > .9, axis=1)

array([False, False, False, ..., False,  True, False])

... kdežto výsledky po sloupcích v našem kontextu nic užitečného neznamenají:

In [37]:
np.any(similarity_table > .9, axis=0)

array([False, False, False, ..., False, False, False])

Výsledek můžeme dokonce sečíst, čímž získáme počet duplikátů:

In [38]:
sum(np.any(similarity_table > .9, axis=1))

305

## Vektorizovaná negace a jiné logické operace

Pro nás je ale možná snazší mít naopak seznam indexů dokumentů, které odstranit *nechceme*, protože na jeho základě pak bude snadné vybudovat pročištěný korpus. Takže reálně chceme předchozí výsledek znegovat. Jenže ouha, operátor `not`, který nás asi napadne, pro tyto účely použít nemůžeme:

In [39]:
not np.any(similarity_table > .9, axis=1)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

Je potěšitelné, že Python raději vyhodí chybu, než aby vyrobil nějakou nesmyslnou hodnotu, ale ta nápověda v tomto kontextu bohužel moc užitečná není -- `any()` už používáme.

Jak tedy na to? Z technických důvodů nelze pro vektorizované logické operace používat běžné logické operátory `not`, `and` a `or`. V této roli je zastupují méně obvyklé operátory `~`, `&` a `|`, které normálně slouží k tzv. *bitwise* operacím, tj. logickým operacím aplikovaným na jednotlivé bity čísel:

```
bitwise not:

~ 101
-----
  010

bitwise and:

  101
& 010
-----
  000

bitwise or:

  101
| 010
-----
  111
```

Z tohoto hlediska je využití těchto operátorů pro vektorizované operace docela intuitivním rozšířením působnosti -- tam se taky operace aplikuje separátně na jednotlivé prvky pole:

In [40]:
arr = np.array([True, False, True])
not_arr = ~arr
arr, not_arr

(array([ True, False,  True]), array([False,  True, False]))

In [41]:
arr & not_arr

array([False, False, False])

In [42]:
arr | not_arr

array([ True,  True,  True])

Vektorizovanou negaci tedy provedeme pomocí operátoru `~`, a znovu si můžeme udělat kontrolní součet -- toto by měla být velikost pročištěného korpusu:

In [43]:
sum(~np.any(similarity_table > .9, axis=1))

3187

## Funkce `numpy.arg*` a dohledávání indexů položek, které splňují nějakou podmínku

Pole pravdivostních hodnot můžeme převést na pole indexů, kde se vyskytuje `True`, např. pomocí funkce `numpy.argwhere()`:

In [44]:
np.argwhere(np.any(similarity_table > .9, axis=1)).flatten()

array([ 103,  128,  180,  209,  285,  342,  356,  490,  602,  637,  649,
        668,  697,  698,  779,  802,  805,  809,  852,  860,  883,  900,
        910,  967,  980,  992, 1018, 1049, 1073, 1079, 1133, 1151, 1180,
       1201, 1203, 1219, 1222, 1230, 1257, 1269, 1280, 1301, 1303, 1318,
       1319, 1320, 1344, 1364, 1376, 1380, 1387, 1391, 1399, 1412, 1427,
       1428, 1438, 1448, 1452, 1463, 1485, 1494, 1499, 1506, 1509, 1520,
       1523, 1531, 1542, 1555, 1568, 1599, 1604, 1638, 1640, 1648, 1657,
       1671, 1676, 1677, 1693, 1697, 1701, 1717, 1720, 1748, 1767, 1772,
       1781, 1788, 1795, 1801, 1805, 1830, 1835, 1838, 1841, 1850, 1853,
       1863, 1893, 1894, 1904, 1907, 1911, 1922, 1939, 1961, 1981, 1996,
       2001, 2005, 2011, 2026, 2027, 2028, 2048, 2052, 2068, 2073, 2088,
       2096, 2120, 2128, 2133, 2151, 2152, 2154, 2161, 2183, 2190, 2192,
       2194, 2196, 2198, 2202, 2206, 2221, 2229, 2246, 2265, 2268, 2271,
       2273, 2289, 2314, 2317, 2319, 2336, 2340, 23

In [45]:
np.argwhere(~np.any(similarity_table > .9, axis=1)).flatten()

array([   0,    1,    2, ..., 3488, 3489, 3491])

Funkcí, jejichž jméno začíná na `arg`, je v `numpy` víc a všechny slouží k tomu, že vrací indexy v poli, pro které platí nějaká podmínka. Např. jestliže funkce `numpy.min` vrátí nejmenší hodnotu v poli, tak funkce `numpy.argmin` mi řekne, který z indexů v poli obsahuje hodnotu, která tuto podmínku splňuje:

In [46]:
arr = np.random.randint(0, 1000, 10)
arr

array([770, 925, 752, 220, 903,  72, 560, 488,  65, 205])

In [47]:
np.min(arr)

65

In [48]:
np.argmin(arr)

8

Konvence pojmenování s prefixem *arg* pochází [z matematiky](https://en.wikipedia.org/wiki/Arg_max). Máme-li funkci $y = f(x)$, tak $\operatorname{min} f(x)$ je minimální hodnota $y$, které tato funkce nabývá, a $\operatorname{arg\,min} f(x)$ je hodnota *argumentu* $x$, pro kterou funkce nabývá oné minimální hodnoty $y$. Platí tedy:

$$\operatorname{min} f(x) = f( \operatorname{arg\,min} f(x) )$$

Když trochu přimhouříme oči, můžeme pole považovat analogicky taky za takovou "funkci" (v matematickém slova smyslu), která pro každou hodnotu indexu ($x$) nabývá hodnotu odpovídající prvku pod tímto indexem uloženého ($y$). Výše uvedenou rovnost tedy můžeme přepsat následovně:

In [49]:
np.min(arr) == arr[np.argmin(arr)]

True

## Finální pročištěný korpus

Nicméně pro vytvoření finálního pročištěného seznamu dokumentů vlastně příslušné indexy ani nepotřebujeme, vystačíme si klidně s původním polem pravdivostních hodnot:

In [50]:
clean_docs = [
    doc
    for doc, is_clean in zip(dirty_docs, ~np.any(similarity_table > .9, axis=1))
    if is_clean
]
assert len(clean_docs) == 3187

# Závěrečné porovnání (čitelnost, rychlost, flexibilita)

Ještě pro zajímavost oba způsoby řešení -- vnořené for-cykly vs. vektorizované operace -- vtělíme do funkcí a srovnáme, jak dlouho jim trvá korpus deduplikovat. Samotné vytvoření objektu `MatrixSimilarity` trvá poměrně dlouho a bylo by v obou funkcích stejné, tak v zájmu rychlejšího a přesnějšího srovnání oběma funkcím předáme již existující výše vytvořený objekt.

In [51]:
def deduplicate(docs, matrix_similarity, verbose=False):
    deduplicated = []
    for i, similarities in enumerate(matrix_similarity):
        for j, similarity in enumerate(similarities):
            if i == j:
                deduplicated.append(docs[i])
                break
            elif similarity > .9:
                if verbose:
                    print(f"{i}, {j}: {similarity}")
                break
    return deduplicated

def deduplicate_vectorized(docs, matrix_similarity):
    similarity_table = np.array(matrix_similarity)
    similarity_table *= np.tri(*similarity_table.shape, -1)
    return [
        doc
        for doc, is_clean in zip(docs, ~np.any(similarity_table > .9, axis=1))
        if is_clean
    ]

Jak vidno, vektorizovaný přístup je nakonec poměrně elegantní a stručný, a pro někoho, kdo se vyzná v manipulaci vícerozměrných polí, i přehledný a srozumitelný. Pro ostatní, kdo se vyznají jen v "normálním" Pythonu, už bohužel méně.

Teď tedy porovnejme rychlost obou variant (zajímá nás údaj *Wall time*, který uvádí čas, který bychom změřili stopkami):

In [52]:
%%time
clean_docs = deduplicate(dirty_docs, matrix_similarity)
assert len(clean_docs) == 3187, f"Expected 3187 documents after deduplication, got {len(clean_docs)}."

CPU times: user 8min 1s, sys: 6min 37s, total: 14min 39s
Wall time: 34.5 s


In [53]:
%%time
clean_docs = deduplicate_vectorized(dirty_docs, matrix_similarity)
assert len(clean_docs) == 3187, f"Expected 3187 documents after deduplication, got {len(clean_docs)}."

CPU times: user 7min 54s, sys: 5min 14s, total: 13min 9s
Wall time: 12.9 s


Na celém korpusu je vektorizovaná funkce tedy skoro 3× rychlejší než funkce využívající vnořené for-cykly.

Zároveň je při porovnání obou funkcí patrná jedna velká výhoda for-cyklového řešení: absolutní flexibilita. V jakémkoli bodě postupu můžu přidat libovolnou operaci, např. průběžné logování pomocí argumentu `verbose`. To do vektorizovaného řešení nevtělím, ani kdybych se rozkrájel.

Ale právě tahle flexibilita je příčinou té pomalosti. Nakonec i všechny ty opakované kontroly podmínky `if verbose: ...` něco stojí -- zkuste ji umazat a znovu pustit test výše, měl by doběhnout o něco málo rychleji.